In [24]:
import numpy as np
import matplotlib
import cv2
from typing import Tuple,Any,List
matplotlib.use('Agg')
import time

Eps = 0.3
BBOX1 = np.array([150,100,200,400])
BBOX2 = np.array([270,100,320,400])

def _neighbors(matrix, a, b):
    neighbors = []
    if b < matrix.shape[1] - 1:
        neighbors.append(matrix[a, b+1])
    if a < matrix.shape[0] - 1:
        neighbors.append(matrix[a+1, b])
    if a > 0:
        neighbors.append(matrix[a-1, b])
    if b > 0:
        neighbors.append(matrix[a, b-1])    
    return neighbors
    

def compute_obervation(label_matrix, image, mean, denominator, cov_inv):
    new_label_matrix = np.zeros((image.shape[0],image.shape[1]))
    for i in range(image.shape[0]-1):
        for j in range(image.shape[1]-1):
            neighbors = _neighbors(label_matrix,i,j)
            factor = 1
            for neighbor in neighbors:
                if neighbor == label_matrix[i,j]:
                    factor = factor*Eps
                else: factor = factor*(1-Eps)
            x_m_1 = image[i,j] - mean[0]
            numerator_1 = np.exp(-(np.dot(np.dot(x_m_1.T,cov_inv[0]),x_m_1)) / 2)
            p1 = ((numerator_1/denominator[0])/100)*factor
            x_m_2 = image[i,j] - mean[1]
            numerator_2 = np.exp(-(np.dot(np.dot(x_m_2.T,cov_inv[1]),x_m_2)) / 2)
            p2 = ((numerator_2/denominator[1])/100)*factor
            p1 = p1/(p1+p2)
            p2 = p2/(p1+p2)
            if p1>=p2:
                new_label_matrix[i,j] = 0
            else:
                new_label_matrix[i,j] = 1
    return new_label_matrix

def gibbs_sampler(image):
    iterations = 100
    area1 = image[BBOX1[0]:BBOX1[2],BBOX1[1]:BBOX1[3]]
    area1 = area1.reshape(area1.shape[0]*area1.shape[1],3)
    area2 = image[BBOX2[0]:BBOX2[2],BBOX2[1]:BBOX2[3]]
    area2 = area2.reshape(area2.shape[0]*area2.shape[1],3)
    m_1 = np.mean(area1, axis = 0)
    m_2 = np.mean(area2, axis = 0)
    s_1 = np.cov(area1,rowvar = False)
    s_2 = np.cov(area2,rowvar = False)
    cov_inverse_1 = np.linalg.inv(s_1)
    cov_inverse_2 = np.linalg.inv(s_2) 
    cov_det_1 = np.linalg.det(s_1)
    cov_det_2 = np.linalg.det(s_2)
    mean = [m_1,m_2]
    cov_inv = [cov_inverse_1,cov_inverse_2]
    cov_det = [cov_det_1,cov_det_2]
    image = cv2.resize(image,(256,256))
    denominator = [np.sqrt(((2 * np.pi)**3) * cov_det[0]),np.sqrt(((2 * np.pi)**3) * cov_det[1])]
    label_matrixx = np.zeros((image.shape[0],image.shape[1]))
    for i in range(image.shape[0]-1):
        for j in range(image.shape[1]-1):
            x_m_1 = image[i,j] - mean[0]
            numerator_1 = np.exp(-(np.dot(np.dot(x_m_1.T,cov_inv[0]),x_m_1)) / 2)
            p1 = ((numerator_1/denominator[0])/100)
            x_m_2 = image[i,j] - mean[1]
            numerator_2 = np.exp(-(np.dot(np.dot(x_m_2.T,cov_inv[1]),x_m_2)) / 2)
            p2 = ((numerator_2/denominator[1])/100)
            if p1>=p2:
                label_matrixx[i,j] = 0
            else:
                label_matrixx[i,j] = 1
    label_matrix = label_matrixx
    results = []
    for iteration in range(iterations):
        print(iteration)
        label_matrix = compute_obervation(label_matrix, image, mean, denominator, cov_inv)
        area1 = image[label_matrix==0]
        area2 = image[label_matrix==1]
        m_1 = np.mean(area1, axis = 0)
        m_2 = np.mean(area2, axis = 0)
        s_1 = np.cov(area1,rowvar = False)
        s_2 = np.cov(area2,rowvar = False)
        cov_inverse_1 = np.linalg.inv(s_1)
        cov_inverse_2 = np.linalg.inv(s_2) 
        cov_det_1 = np.linalg.det(s_1)
        cov_det_2 = np.linalg.det(s_2)
        mean = [m_1,m_2]
        cov_inv = [cov_inverse_1,cov_inverse_2]
        cov_det = [cov_det_1,cov_det_2]
        denominator = [np.sqrt(((2 * np.pi)**3) * cov_det[0]),np.sqrt(((2 * np.pi)**3) * cov_det[1])]
        results.append(label_matrix)
    return label_matrix,results


if __name__ == '__main__':
    image_path = 'D:/laba 1/stock-photo-golden-wheat-field-blue-clear.jpg'
    image = cv2.imread(image_path)
    startt = time.time()
    sg_image, results = gibbs_sampler(image)
    stack_img = np.stack((sg_image,)*3, axis=-1)
    stack_img[:,:,0]*=255
    cv2.imwrite("lab1/result.jpg",stack_img)
    end = time.time()
    print("TIME = ",end-startt)
    for i,mask in enumerate(results):
        stacked_img = np.stack((mask,)*3, axis=-1)*255
        cv2.imwrite(f'D:/laba 1/result/image_{i}.jpg',stack_img)

0


C:\Users\9234~1\AppData\Local\Temp/ipykernel_1380/333067470.py:41: RuntimeWarning: invalid value encountered in double_scalars
  p1 = p1/(p1+p2)


1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
TIME =  71.50763082504272
